In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

Import 1. Branded Food Dataset and then explore it

In [ ]:
branded_food = pd.read_csv("/content/drive/MyDrive/FoodData_Central/branded_food.csv")
branded_food.shape

<ipython-input-5-f3726635a517>:1: DtypeWarning: Columns (2,3,4,6,9,12,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  branded_food = pd.read_csv("/content/drive/MyDrive/FoodData_Central/branded_food.csv")


(1958978, 20)

Generic Function to clean all Dataframes
- drop columns that have more than 50% empty data
- drop duplicate rows

In [ ]:
def clean_dataframe(df):
    duplicates = df[df.duplicated()]
    num_duplicates_removed = duplicates.shape[0]

    df_clean = df.drop_duplicates()

    threshold = len(df_clean) * 0.5
    columns_to_drop = df_clean.columns[df_clean.isnull().sum() > threshold]

    df_clean = df_clean.drop(columns=columns_to_drop)

    df_clean = df_clean.reset_index(drop=True)

    # Print the results
    print(f"Shape before cleaning: {df.shape}")
    print(f"Shape after cleaning: {df_clean.shape}")
    print(f"Number of duplicates found and removed: {num_duplicates_removed}")
    print(f"Columns dropped due to more than 50% missing values: {list(columns_to_drop)} (Total: {len(columns_to_drop)})")
    print("----------")
    missing_values = df_clean.isnull().sum()
    missing_values = missing_values[missing_values > 0]  # Filter for columns with missing values

    if not missing_values.empty:
        print("Columns with missing values and their counts:")
        print(missing_values)
    else:
        print("No missing values found in the cleaned DataFrame.")

    return df_clean

In [ ]:
branded_food_cleaned = clean_dataframe(branded_food)

Shape before cleaning: (1958978, 20)
Shape after cleaning: (1958978, 12)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: ['subbrand_name', 'not_a_significant_source_of', 'household_serving_fulltext', 'package_weight', 'discontinued_date', 'preparation_state_code', 'trade_channel', 'short_description'] (Total: 8)
----------
Columns with missing values and their counts:
brand_owner               17238
brand_name               546806
ingredients                5376
serving_size              10753
serving_size_unit         18993
branded_food_category     10625
modified_date                20
dtype: int64


1. Branded food

Impute null values
- Missing Brand owner, brand name, ingredients imputed with Unknown
- Serving size unit - mode
- Branded food category - Uncategorized
- Serving size - median
- Modified date - with now


In [ ]:
branded_food_cleaned['brand_owner'] = branded_food_cleaned['brand_owner'].fillna('Unknown')
branded_food_cleaned['brand_name'] = branded_food_cleaned['brand_name'].fillna('Unknown')
branded_food_cleaned['ingredients'] = branded_food_cleaned['ingredients'].fillna('Unknown')
branded_food_cleaned['serving_size_unit'] = branded_food_cleaned['serving_size_unit'].fillna(
    branded_food_cleaned['serving_size_unit'].mode()[0]
)
branded_food_cleaned['branded_food_category'] = branded_food_cleaned['branded_food_category'].fillna('Uncategorized')

branded_food_cleaned['serving_size'] = branded_food_cleaned['serving_size'].fillna(
    branded_food_cleaned['serving_size'].median()
)

branded_food_cleaned['modified_date'] = branded_food_cleaned['modified_date'].fillna(pd.to_datetime('now'))

print("Remaining missing values after imputation:")
print(branded_food_cleaned.isna().sum())

Remaining missing values after imputation:
fdc_id                   0
brand_owner              0
brand_name               0
gtin_upc                 0
ingredients              0
serving_size             0
serving_size_unit        0
branded_food_category    0
data_source              0
modified_date            0
available_date           0
market_country           0
dtype: int64


2. Food Update Log Entry

In [ ]:
food_update_log_entry = pd.read_csv("/content/drive/MyDrive/FoodData_Central/food_update_log_entry.csv")
food_update_log_entry_cleaned = clean_dataframe(food_update_log_entry)
food_update_log_entry_cleaned.head()

Shape before cleaning: (1958978, 3)
Shape after cleaning: (1958978, 3)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: [] (Total: 0)
----------
Columns with missing values and their counts:
description    1
dtype: int64


,id,description,last_updated
0,1105904,WESSON Vegetable Oil 1 GAL,2020-11-13
1,1105905,SWANSON BROTH BEEF,2020-11-13
2,1105906,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER,2020-11-13
3,1105907,CAMPBELL'S SLOW KETTLE SOUP CHEESE BROCCOLI,2020-11-13
4,1105908,SWANSON BROTH CHICKEN,2020-11-13


Imputing the missing descriptions with Unknown

In [ ]:
food_update_log_entry_cleaned['description'] = food_update_log_entry_cleaned['description'].fillna('Unknown')
food_update_log_entry_cleaned.isna().sum()

,0
id,0
description,0
last_updated,0


3. Food Attribute Type

In [ ]:
food_attribute_type = pd.read_csv("/content/drive/MyDrive/FoodData_Central/food_attribute_type.csv")
food_attribute_type_cleaned = clean_dataframe(food_attribute_type)
food_attribute_type_cleaned.head()

Shape before cleaning: (5, 3)
Shape after cleaning: (5, 3)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: [] (Total: 0)
----------
No missing values found in the cleaned DataFrame.


,id,name,description
0,998,Update Log,Changes that were made to this food
1,999,Attribute,Generic attributes
2,1000,Common Name,Common names associated with a food.
3,1001,Additional Description,Additional descriptions for the food.
4,1002,Adjustments,"Adjustments made to foods, including moisture ..."


4. Food Attribute

In [ ]:
food_attribute = pd.read_csv("/content/drive/MyDrive/FoodData_Central/food_attribute.csv")
food_attribute_cleaned = clean_dataframe(food_attribute)
food_attribute_cleaned.head()

Shape before cleaning: (2377616, 6)
Shape after cleaning: (2377616, 4)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: ['seq_num', 'food_attribute_type_id'] (Total: 2)
----------
Columns with missing values and their counts:
name    9487
dtype: int64


,id,fdc_id,name,value
0,1009437,1105904,Ingredients,3
1,1009439,1105905,Ingredients,3
2,1009438,1105905,Nutrient Removed,6
3,1009440,1105905,Nutrient Updated,4
4,1009442,1105906,Nutrient Updated,4


Imputing null names with Unknown

In [ ]:
food_attribute_cleaned['name'] = food_attribute_cleaned['name'].fillna('Unknown')
food_attribute_cleaned.isna().sum()

,0
id,0
fdc_id,0
name,0
value,0


5. Food

In [ ]:
food = pd.read_csv("/content/drive/MyDrive/FoodData_Central/food.csv")
food_cleaned = clean_dataframe(food)
food_cleaned.head()

<ipython-input-14-bc396c5570ef>:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  food = pd.read_csv("/content/drive/MyDrive/FoodData_Central/food.csv")


Shape before cleaning: (1958978, 8)
Shape after cleaning: (1958978, 6)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: ['food_category_id', 'trade_channel'] (Total: 2)
----------
Columns with missing values and their counts:
description    1
dtype: int64


,fdc_id,data_type,description,publication_date,market_country,microbe_data
0,1105904,branded_food,WESSON Vegetable Oil 1 GAL,2020-11-13,United States,"[{""min_value"":null,""uom"":null,""microbe_method""..."
1,1105905,branded_food,SWANSON BROTH BEEF,2020-11-13,United States,"[{""min_value"":null,""uom"":null,""microbe_method""..."
2,1105906,branded_food,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER,2020-11-13,United States,"[{""min_value"":null,""uom"":null,""microbe_method""..."
3,1105907,branded_food,CAMPBELL'S SLOW KETTLE SOUP CHEESE BROCCOLI,2020-11-13,United States,"[{""min_value"":null,""uom"":null,""microbe_method""..."
4,1105908,branded_food,SWANSON BROTH CHICKEN,2020-11-13,United States,"[{""min_value"":null,""uom"":null,""microbe_method""..."


Imputing missing descriptions with Unknown

In [ ]:
food_cleaned['description'] = food_cleaned['description'].fillna('Unknown')
food_cleaned.isna().sum()

,0
fdc_id,0
data_type,0
description,0
publication_date,0
market_country,0
microbe_data,0


6. Food Nutrient

In [ ]:
food_nutrient = pd.read_csv("/content/drive/MyDrive/FoodData_Central/food_nutrient.csv")
food_nutrient_cleaned = clean_dataframe(food_nutrient)
food_nutrient_cleaned.head()

Shape before cleaning: (25301180, 11)
Shape after cleaning: (25301180, 5)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: ['data_points', 'min', 'max', 'median', 'footnote', 'min_year_acquired'] (Total: 6)
----------
Columns with missing values and their counts:
derivation_id    830
dtype: int64


,id,fdc_id,nutrient_id,amount,derivation_id
0,13706927,1105904,1257,0.00,71.0
1,13706930,1105904,1293,53.33,71.0
2,13706926,1105904,1253,0.00,75.0
3,13706921,1105904,1092,0.00,75.0
4,13706916,1105904,1008,867.00,71.0


7. Microbe

In [ ]:
microbe = pd.read_csv("/content/drive/MyDrive/FoodData_Central/microbe.csv")
microbe_cleaned = clean_dataframe(microbe)
microbe_cleaned.head()

Shape before cleaning: (13, 7)
Shape after cleaning: (13, 6)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: ['max_value'] (Total: 1)
----------
Columns with missing values and their counts:
uom    2
dtype: int64


,id,foodId,method,microbe_code,min_value,uom
0,1,2331974,AEROBIC_PLATE_COUNT,LACTICASEIBACILLUS_PARACASEI,10000000,NaN
1,2,2332523,AEROBIC_PLATE_COUNT,LACTICASEIBACILLUS_PARACASEI,10000000,NaN
2,3,2336972,AEROBIC_PLATE_COUNT,LACTICASEIBACILLUS_PARACASEI,10000000,X_CFG
3,4,2337521,AEROBIC_PLATE_COUNT,LACTICASEIBACILLUS_PARACASEI,10000000,X_CFG
4,5,2364024,AEROBIC_PLATE_COUNT,LACTICASEIBACILLUS_PARACASEI,10000000,X_CFG


8. Measure Unit

In [ ]:
measure_unit = pd.read_csv("/content/drive/MyDrive/FoodData_Central/measure_unit.csv")
measure_unit_cleaned = clean_dataframe(measure_unit)
measure_unit_cleaned.head()

Shape before cleaning: (122, 2)
Shape after cleaning: (122, 2)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: [] (Total: 0)
----------
No missing values found in the cleaned DataFrame.


,id,name
0,1000,cup
1,1001,tablespoon
2,1002,teaspoon
3,1003,liter
4,1004,milliliter


9. Nurtient

In [ ]:
nutrient = pd.read_csv("/content/drive/MyDrive/FoodData_Central/nutrient.csv")
nutrient_cleaned = clean_dataframe(nutrient)
nutrient_cleaned.head()

Shape before cleaning: (477, 5)
Shape after cleaning: (477, 5)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: [] (Total: 0)
----------
Columns with missing values and their counts:
nutrient_nbr    12
rank            11
dtype: int64


,id,name,unit_name,nutrient_nbr,rank
0,2047,Energy (Atwater General Factors),KCAL,957.0,280.0
1,2048,Energy (Atwater Specific Factors),KCAL,958.0,290.0
2,1001,Solids,G,201.0,200.0
3,1002,Nitrogen,G,202.0,500.0
4,1003,Protein,G,203.0,600.0


In [ ]:
nutrient_cleaned['nutrient_nbr'] = nutrient_cleaned['nutrient_nbr'].fillna(nutrient_cleaned['nutrient_nbr'].mode()[0])
nutrient_cleaned['rank'] = nutrient_cleaned['rank'].fillna(nutrient_cleaned['rank'].mean())
nutrient_cleaned.isna().sum()

,0
id,0
name,0
unit_name,0
nutrient_nbr,0
rank,0


10. Nutrient Incoming Name

In [ ]:
nutrient_incoming_name = pd.read_csv("/content/drive/MyDrive/FoodData_Central/nutrient_incoming_name.csv")
nutrient_incoming_name_cleaned = clean_dataframe(nutrient_incoming_name)
nutrient_incoming_name_cleaned.head()

Shape before cleaning: (730, 3)
Shape after cleaning: (730, 3)
Number of duplicates found and removed: 0
Columns dropped due to more than 50% missing values: [] (Total: 0)
----------
No missing values found in the cleaned DataFrame.


,id,name,nutrient_id
0,1000,NITROGEN-DUMAS METHO,1002
1,1001,Nitrogen,1002
2,1002,NITROGEN-DUMAS METHOD,1002
3,1003,Nitrogen - Kjeldahl,1002
4,1004,Protein,1003


Save all cleaned dataframes back to csv for later use

In [ ]:
import os
def save_dataframes_to_csv(dataframes, folder_path):
    os.makedirs(folder_path, exist_ok=True)

    for df_name, df in dataframes.items():
        file_path = os.path.join(folder_path, f"{df_name}.csv")

        df.to_csv(file_path, index=False)
        print(f"Saved {df_name} to {file_path}")

In [ ]:
dataframes = {
    'branded_food': branded_food_cleaned,
    'nutrient_incoming_name':nutrient_incoming_name_cleaned,
    'nutrient':nutrient_cleaned,
    'microbe':microbe_cleaned,
    'food_nutrient':food_nutrient_cleaned,
    'food':food_cleaned,
    'food_attribute':food_attribute_cleaned,
    'food_attribute_type':food_attribute_type_cleaned,
    'food_update_log_entry':food_update_log_entry_cleaned,
    'measure_unit':measure_unit_cleaned
}

folder_path = '/content/drive/MyDrive/FoodData_Central_Clean'

save_dataframes_to_csv(dataframes, folder_path)

Saved branded_food to /content/drive/MyDrive/FoodData_Central_Clean/branded_food.csv
Saved nutrient_incoming_name to /content/drive/MyDrive/FoodData_Central_Clean/nutrient_incoming_name.csv
Saved nutrient to /content/drive/MyDrive/FoodData_Central_Clean/nutrient.csv
Saved microbe to /content/drive/MyDrive/FoodData_Central_Clean/microbe.csv
Saved food_nutrient to /content/drive/MyDrive/FoodData_Central_Clean/food_nutrient.csv
Saved food to /content/drive/MyDrive/FoodData_Central_Clean/food.csv
Saved food_attribute to /content/drive/MyDrive/FoodData_Central_Clean/food_attribute.csv
Saved food_attribute_type to /content/drive/MyDrive/FoodData_Central_Clean/food_attribute_type.csv
Saved food_update_log_entry to /content/drive/MyDrive/FoodData_Central_Clean/food_update_log_entry.csv
Saved measure_unit to /content/drive/MyDrive/FoodData_Central_Clean/measure_unit.csv
